# Performance dataframes

In [3]:
import os
import re
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

/Users/andersvandvik/Repositories/masters-thesis/venv/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## General parameters

In [4]:
instance_key = 'instance'

avg_constr_obj_key = 'constr obj'
best_obj_key = 'best obj'
worst_obj_key = 'worst obj'
avg_obj_key = 'obj'

max_time_key = 'max time'
min_time_key = 'min time'
avg_time_key = 'time'

max_iter_key = 'max iter'
min_iter_key = 'min iter'
avg_best_sol_iter_key = 'best found iter'
avg_iter_key = 'iter'

set_part_key = 'sp improv (#)'
ls_improv_key = 'ls improv (#)'
best_ls_improv_percent_key = 'best ls improv (%)'
dr_improv_key = 'dr improv (#)'

best_sol_found_by_key = 'best sol found by'

gap_key = 'gap'
preprocess_key = 'preproc'
model_key = 'model time'
variables_key = 'variables'

project_path = os.path.dirname(os.path.abspath('.'))
directory_path_alns = '/output/solstorm/alns/performance/'
directory_path_exact = '/output/solstorm/arcflow/performance/'

generate_df = True
run_number = 'verify'

## Functions

In [5]:
def map_instance_to_data_alns(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
            
            avg_constr_obj = history_json['construction_heuristic_objective']
            obj = history_json['best_objective']
            time = history_json['runtime']
            it = history_json['number_of_iterations']
            best_it = history_json['best_sol_found_in_iteration']
            set_part_improv = history_json['number_of_improvements_by_set_partitioning']
            ls_improv = history_json['number_of_improvements_by_local_search']
            ls_improv_percent = history_json['best_improvement_local_search']
            dr_improv = history_json['number_of_improvements_by_destroy_repair']
            best_sol_found_by = history_json['best_solution_found_by']

            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]

                if obj < data[0]:
                    data[0] = obj
                if obj > data[1]:
                    data[1] = obj
                if time > data[4]:
                    data[4] = time
                if time < data[5]:
                    data[5] = time
                if it > data[7]:
                    data[7] = it
                if it < data[8]:
                    data[8] = it
                
                data[2] += avg_constr_obj
                data[3] += obj
                data[6] += time
                data[9] += best_it
                data[10] += it
                data[11] += set_part_improv
                data[12] += ls_improv
                data[13] += ls_improv_percent
                data[14] += dr_improv
                data[15].append(best_sol_found_by)
                data[16] += 1
            else:
                instance_to_data[instance_name] = [obj, obj, avg_constr_obj, obj, 
                                                   time, time, time, 
                                                   it, it, best_it, it, 
                                                   set_part_improv, ls_improv, ls_improv_percent, dr_improv,
                                                   [best_sol_found_by],
                                                   1]
    return instance_to_data

def generate_run_df_alns(run_name):
    run_path = project_path + directory_path_alns + run_name
    instance_to_data = map_instance_to_data_alns(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               best_obj_key, worst_obj_key, avg_constr_obj_key, avg_obj_key, 
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_best_sol_iter_key, avg_iter_key,
                               set_part_key, ls_improv_key, best_ls_improv_percent_key, dr_improv_key,
                               best_sol_found_by_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        
        nbr_sims = data[16]
        if nbr_sims != 5:
            print(f'{instance} DEVIATES IN SIMULATIONS!')
        
        best_objective = data[0]
        worst_objective = data[1]
        avg_constr_objective = data[2] / nbr_sims
        avg_objective = data[3] / nbr_sims
        max_time = data[4]
        min_time = data[5]
        avg_time = data[6] / nbr_sims
        max_iter = data[7]
        min_iter = data[8]
        avg_best_sol_iter = data[9] / nbr_sims
        avg_iter = data[10] / nbr_sims
        avg_set_part_improv = data[11] / nbr_sims
        avg_ls_improv = data[12] / nbr_sims
        avg_ls_improv_percent = data[13] / nbr_sims
        avg_dr_improv = data[14] / nbr_sims
        best_sol_found_by = data[15]
        
        row = pd.Series({instance_key: instance, 
                         best_obj_key: best_objective,
                         worst_obj_key: worst_objective,
                         avg_constr_obj_key: avg_constr_objective,
                         avg_obj_key: avg_objective,
                         max_time_key: max_time,
                         min_time_key: min_time,
                         avg_time_key: avg_time, 
                         max_iter_key: max_iter,
                         min_iter_key: min_iter,
                         avg_best_sol_iter_key: avg_best_sol_iter,
                         avg_iter_key: avg_iter,
                         set_part_key: avg_set_part_improv,
                         ls_improv_key: avg_ls_improv,
                         best_ls_improv_percent_key: avg_ls_improv_percent,
                         dr_improv_key: avg_dr_improv,
                         best_sol_found_by_key: best_sol_found_by})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_constr_obj_key: df[avg_constr_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_best_sol_iter_key: df[avg_best_sol_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean(),
                          ls_improv_key: df[ls_improv_key].mean(),
                          best_ls_improv_percent_key: df[best_ls_improv_percent_key].mean(),
                          dr_improv_key: df[dr_improv_key].mean(),
                          best_sol_found_by_key: 'Manual'})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def aggregate_df_by_instance_group_alns(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        best_obj = row[best_obj_key]
        worst_obj = row[worst_obj_key]
        avg_constr_obj = row[avg_constr_obj_key]
        avg_obj = row[avg_obj_key]
        max_time = row[max_time_key]
        min_time = row[min_time_key]
        avg_time = row[avg_time_key]
        max_iter = row[max_iter_key]
        min_iter = row[min_iter_key]
        avg_best_sol_iter = row[avg_best_sol_iter_key]
        avg_iter = row[avg_iter_key]
        avg_set_part_improv = row[set_part_key]
        avg_ls_improv = row[ls_improv_key]
        avg_ls_improv_percent = row[best_ls_improv_percent_key]
        avg_dr_improv = row[dr_improv_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += best_obj
            data[1] += worst_obj
            data[2] += avg_constr_obj
            data[3] += avg_obj
            data[4] += max_time
            data[5] += min_time
            data[6] += avg_time
            data[7] += max_iter
            data[8] += min_iter
            data[9] += avg_best_sol_iter
            data[10] += avg_iter
            data[11] += avg_set_part_improv
            data[12] += avg_ls_improv
            data[13] += avg_ls_improv_percent
            data[14] += avg_dr_improv
            data[15] += 1  # Number of times encountered instance size
        else:
            data = [best_obj, worst_obj, avg_constr_obj, avg_obj, 
                    max_time, min_time, avg_time, 
                    max_iter, min_iter, avg_best_sol_iter, avg_iter, 
                    avg_set_part_improv, avg_ls_improv, avg_ls_improv_percent, avg_dr_improv, 1]
            instance_size_to_data[instance_size] = data

    df = pd.DataFrame(columns=['instance_group', 
                               best_obj_key, worst_obj_key, avg_constr_obj_key, avg_obj_key, 
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_best_sol_iter_key, avg_iter_key,
                               set_part_key, ls_improv_key, best_ls_improv_percent_key, dr_improv_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[15]
        if nbr_sims != 5:
            print(f'{instance_size} INSTANCE_SIZE DEVIATES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         best_obj_key: data[0] / nbr_sims,
                         worst_obj_key: data[1] / nbr_sims,
                         avg_constr_obj_key: data[2] / nbr_sims,
                         avg_obj_key: data[3] / nbr_sims, 
                         max_time_key: data[4] / nbr_sims,
                         min_time_key: data[5] / nbr_sims,
                         avg_time_key: data[6] / nbr_sims, 
                         max_iter_key: data[7] / nbr_sims,
                         min_iter_key: data[8] / nbr_sims,
                         avg_best_sol_iter_key: data[9] / nbr_sims,
                         avg_iter_key: data[10] / nbr_sims,
                         set_part_key: data[11] / nbr_sims,
                         ls_improv_key: data[12] / nbr_sims,
                         best_ls_improv_percent_key: data[13] / nbr_sims,
                         dr_improv_key: data[14] / nbr_sims})
    
        df = df.append(row, ignore_index=True)
        
    mean_row = pd.Series({'instance_group': 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_constr_obj_key: df[avg_constr_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_best_sol_iter_key: df[avg_best_sol_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean(),
                          ls_improv_key: df[ls_improv_key].mean(),
                          best_ls_improv_percent_key: df[best_ls_improv_percent_key].mean(),
                          dr_improv_key: df[dr_improv_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def map_instance_to_data_exact(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        with open(run_path + file_name) as file:
            exact_json = json.load(file)

        obj = exact_json['objective']['objective_bound']
        gap = exact_json['objective']['optimality_gap']
        model_runtime = exact_json['runtime']['model_runtime']
        preprocess_runtime = exact_json['runtime']['preprocess_runtime']
        variables = exact_json['variables']['number_of_variables']
        
        if instance_name in instance_to_data:
            print('Multiple versions of same instance!')
        
        instance_to_data[instance_name] = [obj, gap, model_runtime, preprocess_runtime, variables]
    
    return instance_to_data

def generate_run_df_exact(run_name):
    run_path = project_path + directory_path_exact + run_name
    instance_to_data = map_instance_to_data_exact(run_path)

    df = pd.DataFrame(columns=[instance_key, avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    for instance in instance_to_data:
        data = instance_to_data[instance]
        obj = data[0]
        gap = data[1]
        preprocess_runtime = data[2]
        model_runtime = data[3]
        variables = data[4]
        row = pd.Series({instance_key: instance,
                         avg_obj_key: obj,
                         gap_key: gap,
                         preprocess_key: preprocess_runtime,
                         model_key: model_runtime,
                         variables_key: variables})
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(),
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                 max_time_key, min_time_key, 
                 max_iter_key, min_iter_key], 
                axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(5, len(df_total.columns), 5)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

def aggregate_df_by_instance_group_exact(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        obj = row[avg_obj_key]
        gap = row[gap_key]
        preprocess_runtime = row[preprocess_key]
        model_runtime = row[model_key]
        variables = row[variables_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += obj
            data[1] += gap
            data[2] += preprocess_runtime
            data[3] += model_runtime
            data[4] += variables
            data[5] += 1 # Number of times encountered instance size
        else:
            instance_size_to_data[instance_size] = [obj, gap, preprocess_runtime, model_runtime, variables, 1]

    df = pd.DataFrame(columns=['instance_group', avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[5]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         avg_obj_key: data[0] / nbr_sims,
                         gap_key: data[1] / nbr_sims,
                         preprocess_key: data[2] / nbr_sims, 
                         model_key: data[3] / nbr_sims,
                         variables_key: data[4] / nbr_sims})
    
        df = df.append(row, ignore_index=True)

        
    mean_row = pd.Series({'instance_group': 'Mean values', 
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(), 
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def load_df(file_name):
    run_df = pd.read_pickle(f'dataframes/performance/{file_name}')
    # run_df = sort_df(run_df, sort_column)
    return run_df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                 max_time_key, min_time_key, 
                 max_iter_key, min_iter_key, avg_best_sol_iter_key,
                 best_sol_found_by_key], 
                axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    # if drop:
        # li = [i for i in range(5, len(df_total.columns), 5)]
        # df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

## ALNS baseline

In [6]:
if generate_df:
    run_baseline_name = f'{run_number}/testing/'
    run_baseline_df = generate_run_df_alns(run_baseline_name)
    run_baseline_agg_df = aggregate_df_by_instance_group_alns(run_baseline_df)

    run_baseline_file_name = f'dataframes/performance/baseline.pkl'
    run_baseline_agg_file_name = f'dataframes/performance/baseline_agg.pkl'
    run_baseline_df.to_pickle(run_baseline_file_name)
    run_baseline_agg_df.to_pickle(run_baseline_agg_file_name)

13-16-2-1 DEVIATES IN SIMULATIONS!
13-15-2-2 DEVIATES IN SIMULATIONS!
13-18-2-1 DEVIATES IN SIMULATIONS!
11-12-2-1 DEVIATES IN SIMULATIONS!
11-12-2-2 DEVIATES IN SIMULATIONS!
13-15-2-1 DEVIATES IN SIMULATIONS!
13-14-2-1 DEVIATES IN SIMULATIONS!
11-14-2-1 DEVIATES IN SIMULATIONS!
11-15-2-1 DEVIATES IN SIMULATIONS!
11-13-2-1 DEVIATES IN SIMULATIONS!


In [7]:
run_baseline_df = load_df('baseline.pkl')
run_baseline_df

,instance,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#),best sol found by
0,11-12-2-1,3587.5,3587.5,4632.1,3587.5,39.8,39.8,39.8,5000,5000,14.0,5000.0,0.0,3.0,0.1,2.0,[local_search]
1,11-12-2-2,3788.9,3788.9,4537.1,3788.9,51.9,51.9,51.9,5000,5000,399.0,5000.0,1.0,7.0,0.1,4.0,[destroy_repair]
2,11-13-2-1,3470.8,3470.8,3775.7,3470.8,72.2,72.2,72.2,5000,5000,0.0,5000.0,0.0,1.0,0.1,0.0,[local_search]
3,11-14-2-1,3710.8,3710.8,4341.0,3710.8,60.3,60.3,60.3,5000,5000,32.0,5000.0,0.0,5.0,0.1,2.0,[destroy_repair]
4,11-15-2-1,3029.7,3029.7,3751.7,3029.7,26.3,26.3,26.3,5000,5000,51.0,5000.0,0.0,2.0,0.1,4.0,[destroy_repair]
5,13-14-2-1,3681.9,3681.9,4682.3,3681.9,92.0,92.0,92.0,5000,5000,110.0,5000.0,0.0,2.0,0.1,1.0,[local_search]
6,13-15-2-1,4122.6,4122.6,7788.3,4122.6,148.5,148.5,148.5,5000,5000,5.0,5000.0,0.0,3.0,0.2,0.0,[local_search]
7,13-15-2-2,3779.4,3779.4,7035.4,3779.4,56.8,56.8,56.8,5000,5000,25.0,5000.0,0.0,3.0,0.2,2.0,[destroy_repair]
8,13-16-2-1,4239.6,4239.6,5545.6,4239.6,124.4,124.4,124.4,5000,5000,26.0,5000.0,0.0,3.0,0.1,0.0,[local_search]
9,13-18-2-1,4029.1,4029.1,5055.4,4029.1,238.7,238.7,238.7,5000,5000,133.0,5000.0,0.0,3.0,0.2,2.0,[destroy_repair]


In [8]:
run_baseline_agg_df = load_df('baseline_agg.pkl')
run_baseline_agg_df

,instance_group,best obj,worst obj,constr obj,obj,max time,min time,time,max iter,min iter,best found iter,iter,sp improv (#),ls improv (#),best ls improv (%),dr improv (#)
0,11,3517.5,3517.5,4207.5,3517.5,50.1,50.1,50.1,5000.0,5000.0,99.2,5000.0,0.2,3.6,0.1,2.4
1,13,3970.5,3970.5,6021.4,3970.5,132.1,132.1,132.1,5000.0,5000.0,59.8,5000.0,0.0,2.8,0.2,1.0
2,Mean values,3744.0,3744.0,5114.5,3744.0,91.1,91.1,91.1,5000.0,5000.0,79.5,5000.0,0.1,3.2,0.1,1.7


## Sequential ALNS

In [101]:
if generate_df:
    run_sequential_name = 'first/sequential/'
    run_sequential_df = generate_run_df_alns(run_sequential_name)
    run_sequential_agg_df = aggregate_df_by_instance_group_alns(run_sequential_df)

    run_sequential_file_name = f'dataframes/performance/sequential.pkl'
    run_sequential_agg_file_name = f'dataframes/performance/sequential_agg.pkl'
    run_sequential_df.to_pickle(run_sequential_file_name)
    run_sequential_agg_df.to_pickle(run_sequential_agg_file_name)

In [102]:
run_sequential_df = load_df('sequential.pkl')
run_sequential_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.7,0.7,0.7,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.8,0.8,0.8,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.4,1.3,1.3,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.2,1.2,1.2,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.2,1.9,2.1,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,5.6,5.4,5.5,5000,5000,5000.0,0.0
7,7-8-1-3,2071.9,2071.9,2071.9,3.6,3.4,3.5,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.8,4.7,4.8,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,6.7,6.3,6.6,5000,5000,5000.0,0.0


In [103]:
run_sequential_agg_df = load_df('sequential_agg.pkl')
run_sequential_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,0.9,0.9,5000.0,5000.0,5000.0,0.0
1,7,2093.3,2093.3,2093.3,4.6,4.3,4.5,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,10.8,9.5,10.1,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,29.9,26.0,28.0,5000.0,5000.0,5000.0,0.0
4,13,3970.5,3970.5,3970.5,67.4,56.4,61.7,5000.0,5000.0,5000.0,0.0
5,15,6179.0,8399.9,7944.4,86.7,74.9,80.7,5000.0,5000.0,5000.0,0.0
6,17,4827.9,4986.8,4943.2,151.6,124.0,138.8,5000.0,5000.0,5000.0,0.0
7,19,4926.9,5173.1,5095.8,190.6,166.2,178.9,5000.0,5000.0,5000.0,0.0
8,21,8362.7,9332.5,9116.9,238.3,201.3,218.0,5000.0,5000.0,5000.0,0.0
9,23,5619.4,5995.6,5871.9,361.2,326.2,342.6,5000.0,5000.0,5000.0,0.0


## LNS

In [104]:
if generate_df:
    run_lns_name = f'{run_number}/lns/'
    run_lns_df = generate_run_df_alns(run_lns_name)
    run_lns_agg_df = aggregate_df_by_instance_group_alns(run_lns_df)

    run_lns_file_name = f'dataframes/performance/lns.pkl'
    run_lns_agg_file_name = f'dataframes/performance/lns_agg.pkl'
    run_lns_df.to_pickle(run_lns_file_name)
    run_lns_agg_df.to_pickle(run_lns_agg_file_name)

11-12-2-2 HAS LESS THAN FIVE SIMULATIONS!


In [105]:
run_lns_df = load_df('lns.pkl')
run_lns_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.7,0.7,0.7,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.9,0.8,0.8,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.3,1.2,1.3,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.2,1.1,1.1,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.0,1.7,1.8,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,1.8,1.6,1.7,5000,5000,5000.0,0.0
7,7-8-1-3,2080.2,2080.2,2080.2,2.0,1.9,1.9,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.4,4.1,4.3,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,2.7,2.2,2.4,5000,5000,5000.0,0.0


In [106]:
run_lns_agg_df = load_df('lns_agg.pkl')
run_lns_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,0.9,0.9,5000.0,5000.0,5000.0,0.0
1,7,2095.0,2095.0,2095.0,2.6,2.3,2.4,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,5.3,4.1,4.6,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3366.0,19.8,17.3,18.0,5000.0,5000.0,4800.0,0.0
4,13,3970.5,3970.5,3970.5,34.8,30.4,32.3,5000.0,5000.0,5000.0,0.0
5,15,8375.4,8427.9,8400.4,40.1,34.1,37.6,5000.0,5000.0,5000.0,0.0
6,17,4966.1,4990.1,4974.4,60.5,51.8,55.8,5000.0,5000.0,5000.0,0.0
7,19,5153.5,5185.8,5166.5,77.6,66.5,71.4,5000.0,5000.0,5000.0,0.0
8,21,9298.6,9360.3,9327.6,87.5,76.0,82.9,5000.0,5000.0,5000.0,0.0
9,23,5959.4,6032.0,5983.1,134.5,118.0,126.7,5000.0,5000.0,5000.0,0.0


## ALNS + local search

In [107]:
if generate_df:
    run_ls_name = f'{run_number}/ls/'
    run_ls_df = generate_run_df_alns(run_ls_name)
    run_ls_agg_df = aggregate_df_by_instance_group_alns(run_ls_df)

    run_ls_file_name = f'dataframes/performance/ls.pkl'
    run_ls_agg_file_name = f'dataframes/performance/ls_agg.pkl'
    run_ls_df.to_pickle(run_ls_file_name)
    run_ls_agg_df.to_pickle(run_ls_agg_file_name)

In [108]:
run_ls_df = load_df('ls.pkl')
run_ls_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,1.6,1.5,1.5,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,1.4,1.3,1.3,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,1.6,1.5,1.6,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,2.3,2.2,2.2,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.9,1.8,1.9,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,4.5,4.3,4.4,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,4.3,3.5,4.0,5000,5000,5000.0,0.0
7,7-8-1-3,2080.2,2080.2,2080.2,4.7,4.6,4.6,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,7.4,7.1,7.3,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,5.7,5.1,5.2,5000,5000,5000.0,0.0


In [109]:
run_ls_agg_df = load_df('ls_agg.pkl')
run_ls_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.8,1.7,1.7,5000.0,5000.0,5000.0,0.0
1,7,2095.0,2095.0,2095.0,5.3,4.9,5.1,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,9.3,8.2,8.6,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,55.4,49.3,52.5,5000.0,5000.0,5000.0,0.0
4,13,3970.5,3970.5,3970.5,96.7,85.4,91.5,5000.0,5000.0,5000.0,0.0
5,15,8375.4,8395.9,8383.1,98.6,82.2,92.5,5000.0,5000.0,5000.0,0.0
6,17,4966.1,4971.6,4967.6,197.8,169.9,186.6,5000.0,5000.0,5000.0,0.0
7,19,5153.5,5164.7,5156.6,265.6,240.4,252.7,5000.0,5000.0,5000.0,0.0
8,21,9295.9,9335.4,9315.2,228.7,196.8,211.8,5000.0,5000.0,5000.0,0.0
9,23,5963.5,5996.3,5971.7,453.1,425.4,440.7,4902.0,4753.8,4826.0,0.0


## ALNS + set partitioning

In [110]:
if generate_df: 
    run_sp_name = f'{run_number}/sp/'
    run_sp_df = generate_run_df_alns(run_sp_name)
    run_sp_agg_df = aggregate_df_by_instance_group_alns(run_sp_df)

    run_sp_file_name = f'dataframes/performance/sp.pkl'
    run_sp_agg_file_name = f'dataframes/performance/sp_agg.pkl'
    run_sp_df.to_pickle(run_sp_file_name)
    run_sp_agg_df.to_pickle(run_sp_agg_file_name)

21-26-3-1 HAS LESS THAN FIVE SIMULATIONS!


In [111]:
run_sp_df = load_df('sp.pkl')
run_sp_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.8,0.8,0.8,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,1.0,0.9,0.9,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.4,1.3,1.4,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.3,1.3,1.3,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.1,1.8,2.0,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,2.1,1.8,1.9,5000,5000,5000.0,0.0
7,7-8-1-3,2080.2,2080.2,2080.2,2.2,2.1,2.2,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,5.3,4.7,5.0,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,3.3,2.5,2.9,5000,5000,5000.0,0.0


In [112]:
run_sp_agg_df = load_df('sp_agg.pkl')
run_sp_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,1.0,1.0,5000.0,5000.0,5000.0,0.0
1,7,2095.0,2095.0,2095.0,3.0,2.6,2.8,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,5.9,5.1,5.5,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,24.9,21.5,23.1,5000.0,5000.0,5000.0,0.0
4,13,3970.5,3970.5,3970.5,47.2,41.1,44.5,5000.0,5000.0,5000.0,0.0
5,15,8375.4,8394.5,8381.2,59.5,47.0,53.7,5000.0,5000.0,5000.0,0.8
6,17,4966.1,4975.0,4969.5,100.0,77.7,89.0,5000.0,5000.0,5000.0,1.3
7,19,5153.5,5163.0,5157.0,150.1,124.4,137.0,5000.0,5000.0,5000.0,1.9
8,21,9260.5,9316.5,8718.1,180.3,146.1,157.4,5000.0,5000.0,4800.0,2.6
9,23,5959.4,6001.5,5971.6,344.6,262.2,294.3,5000.0,5000.0,5000.0,2.4


## ALNS + local search + set partitioning

In [113]:
if generate_df:
    run_lssp_name = f'{run_number}/lssp/'
    run_lssp_df = generate_run_df_alns(run_lssp_name)
    run_lssp_agg_df = aggregate_df_by_instance_group_alns(run_lssp_df)

    run_lssp_file_name = f'dataframes/performance/lssp.pkl'
    run_lssp_agg_file_name = f'dataframes/performance/lssp_agg.pkl'
    run_lssp_df.to_pickle(run_lssp_file_name)
    run_lssp_agg_df.to_pickle(run_lssp_agg_file_name)

25-33-4-1 HAS LESS THAN FIVE SIMULATIONS!


In [114]:
run_lssp_df = load_df('lssp.pkl')
run_lssp_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11-12-2-1,3587.5,3587.5,3587.5,57.4,50.9,54.8,5000,5000,5000.0,0.0
1,11-12-2-2,3788.9,3788.9,3788.9,65.0,59.0,62.6,5000,5000,5000.0,0.0
2,11-13-2-1,3470.8,3470.8,3470.8,82.6,71.7,78.3,5000,5000,5000.0,0.0
3,11-14-2-1,3710.8,3710.8,3710.8,82.0,76.1,78.9,5000,5000,5000.0,0.0
4,11-15-2-1,3029.7,3029.7,3029.7,51.3,44.5,47.8,5000,5000,5000.0,0.0
5,13-14-2-1,3681.9,3681.9,3681.9,133.6,117.6,123.2,5000,5000,5000.0,0.0
6,13-15-2-1,4122.6,4122.6,4122.6,126.1,114.1,121.0,5000,5000,5000.0,0.2
7,13-15-2-2,3779.4,3779.4,3779.4,80.7,66.9,71.9,5000,5000,5000.0,0.2
8,13-16-2-1,4239.6,4239.6,4239.6,146.5,123.3,134.4,5000,5000,5000.0,0.0
9,13-18-2-1,4029.1,4029.1,4029.1,302.7,226.7,252.2,5000,5000,5000.0,0.2


In [115]:
run_lssp_agg_df = load_df('lssp_agg.pkl')
run_lssp_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11,3517.5,3517.5,3517.5,67.7,60.4,64.5,5000.0,5000.0,5000.0,0.0
1,13,3970.5,3970.5,3970.5,157.9,129.7,140.5,5000.0,5000.0,5000.0,0.1
2,15,8375.4,8375.4,8375.4,161.5,123.8,146.8,5000.0,5000.0,5000.0,0.8
3,17,4966.1,4968.5,4966.6,479.5,386.1,441.6,4698.6,4631.0,4663.5,0.9
4,19,5153.5,5157.8,5154.4,505.8,501.8,503.5,3075.4,2817.2,2951.4,1.6
5,21,9186.8,9267.3,9240.5,531.1,405.2,479.6,4656.8,3960.4,4250.4,2.6
6,23,5959.4,5965.2,5962.3,601.6,600.2,600.7,2375.0,1826.0,2092.1,2.2
7,25,6629.6,6647.9,6063.6,603.6,600.1,553.0,1988.4,1722.2,1766.2,2.0
8,Mean values,5969.9,5983.8,5906.4,388.6,350.9,366.3,3974.3,3744.6,3840.5,1.3


## Exact solver

In [116]:
if generate_df:
    run_exact_name = 'results/'
    run_exact_df = generate_run_df_exact(run_exact_name)
    run_exact_agg_df = aggregate_df_by_instance_group_exact(run_exact_df)

    run_exact_file_name = f'dataframes/performance/exact.pkl'
    run_exact_agg_file_name = f'dataframes/performance/exact_agg.pkl'
    run_exact_df.to_pickle(run_exact_file_name)
    run_exact_agg_df.to_pickle(run_exact_agg_file_name)

21 INSTANCE_SIZE LESS THAN FIVE INSTANCES!


In [117]:
run_exact_df = load_df('exact.pkl')
run_exact_df

,instance,obj,gap,preproc,model time,variables
0,5-5-1-1,2519.9,0.0,1.3,1.5,24808
1,5-5-1-2,2214.2,0.0,3.7,1.5,36406
2,5-6-1-1,1939.6,0.0,4.2,1.8,40610
3,5-7-1-1,2709.7,0.0,2.7,2.3,35034
4,5-7-1-2,1704.6,0.0,16.0,2.7,64346
5,7-8-1-1,2046.9,0.0,15.2,3.5,71164
6,7-8-1-2,1971.3,0.0,65.4,3.7,84072
7,7-8-1-3,2071.9,0.0,9.1,3.0,47880
8,7-9-1-1,2170.4,0.0,26.6,3.5,80958
9,7-9-1-2,2206.1,0.0,38.7,3.7,85362


In [118]:
run_exact_agg_df = load_df('exact_agg.pkl')
run_exact_agg_df

,instance_group,obj,gap,preproc,model time,variables
0,5,2217.6,0.0,5.6,2.0,40240.8
1,7,2093.3,0.0,31.0,3.5,73887.2
2,9,5627.5,0.0,43.7,5.7,124820.4
3,11,3450.2,0.0,2015.2,13.7,309915.6
4,13,3717.5,0.1,3493.6,20.3,447488.4
5,15,7257.5,0.1,3231.8,27.4,606759.0
6,17,3677.1,0.4,3600.8,45.0,984075.2
7,19,3877.3,0.8,3601.8,52.3,1135832.8
8,21,5603.5,0.6,3600.3,61.7,1344890.0
9,Mean values,4169.1,0.2,2180.4,25.7,563101.0


## Parallel vs. sequential heuristics

In [119]:
baseline_sequential_df = merge_dfs([run_baseline_df, run_sequential_df], True)
baseline_sequential_agg_df = merge_dfs([run_baseline_agg_df, run_sequential_agg_df], True)

In [120]:
baseline_sequential_df

,instance,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5-5-1-1,2519.9,0.7,5000.0,0.0,2519.9,0.7,5000.0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.0,2214.2,0.7,5000.0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,1939.6,0.8,5000.0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,0.0,2709.7,1.3,5000.0,0.0
4,5-7-1-2,1704.6,1.2,5000.0,0.0,1704.6,1.2,5000.0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,0.0,2046.9,2.1,5000.0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,0.0,1971.3,5.5,5000.0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,0.0,2071.9,3.5,5000.0,0.0
8,7-9-1-1,2170.4,4.3,5000.0,0.0,2170.4,4.8,5000.0,0.0
9,7-9-1-2,2206.1,2.3,5000.0,0.0,2206.1,6.6,5000.0,0.0


In [121]:
baseline_sequential_agg_df

,instance_group,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5,2217.6,1.0,5000.0,0.0,2217.6,0.9,5000.0,0.0
1,7,2095.0,2.4,5000.0,0.0,2093.3,4.5,5000.0,0.0
2,9,5627.5,4.5,5000.0,0.0,5627.5,10.1,5000.0,0.0
3,11,3517.5,18.8,5000.0,0.0,3517.5,28.0,5000.0,0.0
4,13,3970.5,32.5,5000.0,0.0,3970.5,61.7,5000.0,0.0
5,15,8389.0,37.3,5000.0,0.0,7944.4,80.7,5000.0,0.0
6,17,4976.7,57.9,5000.0,0.0,4943.2,138.8,5000.0,0.0
7,19,5167.4,72.6,5000.0,0.0,5095.8,178.9,5000.0,0.0
8,21,9118.2,80.3,4800.0,0.0,9116.9,218.0,5000.0,0.0
9,23,5983.2,130.5,5000.0,0.0,5871.9,342.6,5000.0,0.0


## ALNS vs. LNS

In [122]:
baseline_lns_df = merge_dfs([run_baseline_df, run_lns_df], True)
baseline_lns_agg_df = merge_dfs([run_baseline_agg_df, run_lns_agg_df], True)

In [123]:
baseline_lns_df

,instance,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5-5-1-1,2519.9,0.7,5000.0,0.0,2519.9,0.7,5000.0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.0,2214.2,0.7,5000.0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,1939.6,0.8,5000.0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,0.0,2709.7,1.3,5000.0,0.0
4,5-7-1-2,1704.6,1.2,5000.0,0.0,1704.6,1.1,5000.0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,0.0,2046.9,1.8,5000.0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,0.0,1971.3,1.7,5000.0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,0.0,2080.2,1.9,5000.0,0.0
8,7-9-1-1,2170.4,4.3,5000.0,0.0,2170.4,4.3,5000.0,0.0
9,7-9-1-2,2206.1,2.3,5000.0,0.0,2206.1,2.4,5000.0,0.0


In [124]:
baseline_lns_agg_df

,instance_group,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5,2217.6,1.0,5000.0,0.0,2217.6,0.9,5000.0,0.0
1,7,2095.0,2.4,5000.0,0.0,2095.0,2.4,5000.0,0.0
2,9,5627.5,4.5,5000.0,0.0,5627.5,4.6,5000.0,0.0
3,11,3517.5,18.8,5000.0,0.0,3366.0,18.0,4800.0,0.0
4,13,3970.5,32.5,5000.0,0.0,3970.5,32.3,5000.0,0.0
5,15,8389.0,37.3,5000.0,0.0,8400.4,37.6,5000.0,0.0
6,17,4976.7,57.9,5000.0,0.0,4974.4,55.8,5000.0,0.0
7,19,5167.4,72.6,5000.0,0.0,5166.5,71.4,5000.0,0.0
8,21,9118.2,80.3,4800.0,0.0,9327.6,82.9,5000.0,0.0
9,23,5983.2,130.5,5000.0,0.0,5983.1,126.7,5000.0,0.0


## ALNS vs. ALNS + local search

In [125]:
baseline_ls_df = merge_dfs([run_baseline_df, run_ls_df], True)
baseline_ls_agg_df = merge_dfs([run_baseline_agg_df, run_ls_agg_df], True)

In [126]:
baseline_ls_df

,instance,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5-5-1-1,2519.9,0.7,5000.0,0.0,2519.9,1.5,5000.0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.0,2214.2,1.3,5000.0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,1939.6,1.6,5000.0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,0.0,2709.7,2.2,5000.0,0.0
4,5-7-1-2,1704.6,1.2,5000.0,0.0,1704.6,1.9,5000.0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,0.0,2046.9,4.4,5000.0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,0.0,1971.3,4.0,5000.0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,0.0,2080.2,4.6,5000.0,0.0
8,7-9-1-1,2170.4,4.3,5000.0,0.0,2170.4,7.3,5000.0,0.0
9,7-9-1-2,2206.1,2.3,5000.0,0.0,2206.1,5.2,5000.0,0.0


In [127]:
baseline_ls_agg_df

,instance_group,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5,2217.6,1.0,5000.0,0.0,2217.6,1.7,5000.0,0.0
1,7,2095.0,2.4,5000.0,0.0,2095.0,5.1,5000.0,0.0
2,9,5627.5,4.5,5000.0,0.0,5627.5,8.6,5000.0,0.0
3,11,3517.5,18.8,5000.0,0.0,3517.5,52.5,5000.0,0.0
4,13,3970.5,32.5,5000.0,0.0,3970.5,91.5,5000.0,0.0
5,15,8389.0,37.3,5000.0,0.0,8383.1,92.5,5000.0,0.0
6,17,4976.7,57.9,5000.0,0.0,4967.6,186.6,5000.0,0.0
7,19,5167.4,72.6,5000.0,0.0,5156.6,252.7,5000.0,0.0
8,21,9118.2,80.3,4800.0,0.0,9315.2,211.8,5000.0,0.0
9,23,5983.2,130.5,5000.0,0.0,5971.7,440.7,4826.0,0.0


## ALNS vs. ALNS + set partitioning

In [128]:
baseline_sp_df = merge_dfs([run_baseline_df, run_sp_df], True)
baseline_sp_agg_df = merge_dfs([run_baseline_agg_df, run_sp_agg_df], True)

In [129]:
baseline_sp_df

,instance,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5-5-1-1,2519.9,0.7,5000.0,0.0,2519.9,0.8,5000.0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.0,2214.2,0.7,5000.0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,1939.6,0.9,5000.0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,0.0,2709.7,1.4,5000.0,0.0
4,5-7-1-2,1704.6,1.2,5000.0,0.0,1704.6,1.3,5000.0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,0.0,2046.9,2.0,5000.0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,0.0,1971.3,1.9,5000.0,0.0
7,7-8-1-3,2080.2,2.0,5000.0,0.0,2080.2,2.2,5000.0,0.0
8,7-9-1-1,2170.4,4.3,5000.0,0.0,2170.4,5.0,5000.0,0.0
9,7-9-1-2,2206.1,2.3,5000.0,0.0,2206.1,2.9,5000.0,0.0


In [130]:
baseline_sp_agg_df

,instance_group,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5,2217.6,1.0,5000.0,0.0,2217.6,1.0,5000.0,0.0
1,7,2095.0,2.4,5000.0,0.0,2095.0,2.8,5000.0,0.0
2,9,5627.5,4.5,5000.0,0.0,5627.5,5.5,5000.0,0.0
3,11,3517.5,18.8,5000.0,0.0,3517.5,23.1,5000.0,0.0
4,13,3970.5,32.5,5000.0,0.0,3970.5,44.5,5000.0,0.0
5,15,8389.0,37.3,5000.0,0.0,8381.2,53.7,5000.0,0.8
6,17,4976.7,57.9,5000.0,0.0,4969.5,89.0,5000.0,1.3
7,19,5167.4,72.6,5000.0,0.0,5157.0,137.0,5000.0,1.9
8,21,9118.2,80.3,4800.0,0.0,8718.1,157.4,4800.0,2.6
9,23,5983.2,130.5,5000.0,0.0,5971.6,294.3,5000.0,2.4


## ALNS vs. ALNS + local search + set partitioning

In [131]:
baseline_lssp_df = merge_dfs([run_baseline_df, run_lssp_df], True)
baseline_lssp_agg_df = merge_dfs([run_baseline_agg_df, run_lssp_agg_df], True)

In [132]:
baseline_lssp_df

,instance,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5-5-1-1,2519.9,0.7,5000.0,0.0,3587.5,54.8,5000.0,0.0
1,5-5-1-2,2214.2,0.7,5000.0,0.0,3788.9,62.6,5000.0,0.0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,3470.8,78.3,5000.0,0.0
3,5-7-1-1,2709.7,1.3,5000.0,0.0,3710.8,78.9,5000.0,0.0
4,5-7-1-2,1704.6,1.2,5000.0,0.0,3029.7,47.8,5000.0,0.0
5,7-8-1-1,2046.9,1.8,5000.0,0.0,3681.9,123.2,5000.0,0.0
6,7-8-1-2,1971.3,1.8,5000.0,0.0,4122.6,121.0,5000.0,0.2
7,7-8-1-3,2080.2,2.0,5000.0,0.0,3779.4,71.9,5000.0,0.2
8,7-9-1-1,2170.4,4.3,5000.0,0.0,4239.6,134.4,5000.0,0.0
9,7-9-1-2,2206.1,2.3,5000.0,0.0,4029.1,252.2,5000.0,0.2


In [133]:
baseline_lssp_agg_df

,instance_group,obj,time,iter,sp improv,obj,time,iter,sp improv
0,5,2217.6,1.0,5000.0,0.0,3517.5,64.5,5000.0,0.0
1,7,2095.0,2.4,5000.0,0.0,3970.5,140.5,5000.0,0.1
2,9,5627.5,4.5,5000.0,0.0,8375.4,146.8,5000.0,0.8
3,11,3517.5,18.8,5000.0,0.0,4966.6,441.6,4663.5,0.9
4,13,3970.5,32.5,5000.0,0.0,5154.4,503.5,2951.4,1.6
5,15,8389.0,37.3,5000.0,0.0,9240.5,479.6,4250.4,2.6
6,17,4976.7,57.9,5000.0,0.0,5962.3,600.7,2092.1,2.2
7,19,5167.4,72.6,5000.0,0.0,6063.6,553.0,1766.2,2.0
8,21,9118.2,80.3,4800.0,0.0,5906.4,366.3,3840.5,1.3
9,23,5983.2,130.5,5000.0,0.0,NaN,NaN,NaN,NaN
